In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys 
from torch.nn import functional as F
import configparser
import pandas as pd 

import numpy as np
import torch
import matplotlib.pyplot as plt

import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
from os.path import join as oj
sys.path.insert(0, "../src")
from unet import UNet
import my_data
import pickle as pkl
pd.set_option('display.float_format', lambda x: '%.3f' % x)
cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")
config = configparser.ConfigParser()
config.read('../config.ini');
data_path =config['DATASET']['data_path'] 
fig_path =config['PATHS']['figure_path'] 


In [ ]:
config['PATHS']['model_path'] 

In [ ]:
# need exp with no active learning
# need exp with prop increasingf
fnames = sorted([oj(config['PATHS']['model_path'] , fname) for fname in os.listdir(config['PATHS']['model_path'] ) if "pkl" in fname]) 
results_list = [pd.Series(pkl.load(open(fname, "rb"))) for fname in (fnames)] 
results = pd.concat(results_list, axis=1).T.infer_objects()


results.reset_index(inplace=True)
results[['foldername',  'cost_function', 'add_ratio',  'test_dice_score_all', 
         'experiment_name']].groupby([ 'experiment_name', 'cost_function','add_ratio', 
         'foldername' ]).mean()



In [ ]:
# make figure barplot
results_dict ={}
results_dict['Base model'] = results[results.experiment_name == 'NoActiveLearning'].test_dice_score
results_dict['Random'] = results[results.experiment_name == 'ActiveLearning'][results.cost_function == 'random_cost'][results.add_ratio == .02].test_dice_score

results_dict['EMC'] = results[results.experiment_name == 'ActiveLearning'][results.cost_function == 'emc'][results.add_ratio == .02].test_dice_score
results_dict['Active Learning'] = results[results.experiment_name == 'ActiveLearning'][results.cost_function == 'cut_off_cost'][results.add_ratio == .02].test_dice_score
results_dict['Full Dataset'] = results[results.experiment_name == 'FullDatasetTraining'].test_dice_score
# results_dict['Human repeat'] = np.asarray([0.9761262536048889])
for key in results_dict:
    results_dict[key] = np.asarray(results_dict[key])
# make barplot
plt.figure(figsize=(10,5))
plt.bar(results_dict.keys(), 
        [np.mean(results_dict[key]) for key in results_dict.keys()], 
        yerr=[np.std(results_dict[key])/np.sqrt(len(results_dict[key])) for key in results_dict.keys()],
        # color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
        color = ['g', 'g','g', 'r','g',])


plt.xticks(fontsize=30, rotation=20)
plt.axhline(0.965, color='k', linestyle='--', label ='Human Repeat',)
plt.annotate('Human Repeat', xy=(0, 0.91), xytext=(0, 0.977), 
             arrowprops=dict(facecolor='black', arrowstyle='->'), fontsize=20)

plt.ylim(0.85, 0.98)
plt.ylabel('Dice Score', fontsize=25)

# plt.legend(loc = 'center left', fontsize=20) 
plt.savefig(oj(fig_path, 'results_dice.pdf'), dpi=300, bbox_inches='tight')

In [ ]:
import seaborn as sns
#lineplot
results_here = results[results.experiment_name == 'ActiveLearning']


results_here_copy = results[results.experiment_name == 'NoActiveLearning']
results_here_copy.cost_function = 'cut_off_cost'
results_here = pd.concat([results_here, results_here_copy])
results_here_copy = results[results.experiment_name == 'NoActiveLearning']
results_here_copy.cost_function = 'random_cost'
results_here = pd.concat([results_here, results_here_copy])
results_here_copy = results[results.experiment_name == 'NoActiveLearning']
results_here_copy.cost_function = 'emc'
results_here = pd.concat([results_here, results_here_copy])



plt.figure(figsize=(10,5))
#rename 
results_here.cost_function = results_here.cost_function.replace('cut_off_cost', 'Uncertainty')
results_here.cost_function = results_here.cost_function.replace('random_cost', 'Random')

results_here.cost_function = results_here.cost_function.replace('emc', 'EMC')
#make matplotlib
# results_here = results_here[results_here.add_ratio < .1]
plt.figure(figsize=(10,5))

#rename column
results_here = results_here.rename(columns={'test_dice_score': 'Dice Score'})
results_here.add_ratio = results_here.add_ratio*100
results_here = results_here.rename (columns={'add_ratio': 'Percentage of data'})

results_here = results_here.rename(columns={'cost_function': 'Cost function'})
# results_here = results_here.rename(columns={'add_ratio': '% of Dataset'})

#increase font size for seaborm
sns.set(font_scale=2)
sns.set_style("whitegrid")
# sns.despine(offset=10, trim=True);
# g_results = sns.lineplot(data=results_here, x="% of Dataset", y="Dice Score", hue="Cost function", errorbar='se', palette="tab10", linewidth=2.5)

g_results = sns.lineplot(data=results_here, x="Percentage of data", y="Dice Score", hue="Cost function", errorbar='se', palette="tab10", linewidth=2.5)

plt.ylim(.85, .98)
plt.axhline(0.968, color='k', linestyle='--', label ='Full dataset',)
# an arrow that points to the h line and says "Full dataset"
plt.annotate('Full dataset', xy=(0, 0.968), xytext=(0, 0.975), arrowprops=dict(facecolor='black', shrink=0.1),)
# savefig
plt.savefig(oj(fig_path, 'results_lineplot.pdf'), dpi=300, bbox_inches='tight')

In [ ]:
import seaborn as sns
#lineplot
results_here = results[results.experiment_name == 'ActiveLearning']
# results_here = results_here[results_here.add_ratio < .1]

results_here_copy = results[results.experiment_name == 'NoActiveLearning']
results_here_copy.cost_function = 'cut_off_cost'
results_here = pd.concat([results_here, results_here_copy])
results_here_copy = results[results.experiment_name == 'NoActiveLearning']
results_here_copy.cost_function = 'random_cost'
results_here = pd.concat([results_here, results_here_copy])
results_here_copy = results[results.experiment_name == 'NoActiveLearning']
results_here_copy.cost_function = 'emc'
results_here = pd.concat([results_here, results_here_copy])



plt.figure(figsize=(10,5))
#rename 
results_here.cost_function = results_here.cost_function.replace('cut_off_cost', 'Uncertainty')
results_here.cost_function = results_here.cost_function.replace('random_cost', 'Random')

results_here.cost_function = results_here.cost_function.replace('emc', 'EMC')
#make matplotlib
# results_here = results_here[results_here.add_ratio < .1]
plt.figure(figsize=(10,5))

#rename column
results_here = results_here.rename(columns={'test_dice_score_all': 'Dice Score (All)'})
results_here.add_ratio = results_here.add_ratio*100
results_here = results_here.rename (columns={'add_ratio': 'Percentage of data'})

results_here = results_here.rename(columns={'cost_function': 'Cost function'})
# results_here = results_here.rename(columns={'add_ratio': '% of Dataset'})

#increase font size for seaborm
sns.set(font_scale=2)
sns.set_style("whitegrid")
# sns.despine(offset=10, trim=True);
# g_results = sns.lineplot(data=results_here, x="% of Dataset", y="Dice Score", hue="Cost function", errorbar='se', palette="tab10", linewidth=2.5)

g_results = sns.lineplot(data=results_here, x="Percentage of data", y="Dice Score (All)", hue="Cost function", errorbar='se', palette="tab10", linewidth=2.5)

plt.ylim(.85, .99)
plt.axhline(0.978, color='k', linestyle='--', label ='Full dataset',)
# an arrow that points to the h line and says "Full dataset"
plt.annotate('Full dataset', xy=(0, 0.978), xytext=(0, 0.97), arrowprops=dict(facecolor='black', shrink=0.1),)
# savefig
plt.savefig(oj(fig_path, 'results_lineplot_all.pdf'), dpi=300, bbox_inches='tight')